# Aula 12: Funções Customizadas e Otimização com Fusion Kernel

### Nome: Luca Mizrahi

In [1]:
!nvidia-smi

Fri Sep 20 11:04:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%writefile saxpy_functor.cu

#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <iostream>

// Definição do functor Saxpy
struct saxpy {
    int a;  // Constante 'a'

    // Construtor que inicializa 'a'
    saxpy(int a_) : a(a_) {}

    // Definindo o operador () para que a struct se comporte como uma função
    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;
    }
};

Writing saxpy_functor.cu


#### *Introdução a Operações customizadas*

In [3]:
%%writefile saxpy_exercicio.cu

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <cstdlib> // Para gerar números aleatórios

// Definição do functor Saxpy
struct saxpy {
    int a;  // Constante 'a'

    // Construtor que inicializa 'a'
    saxpy(int a_) : a(a_) {}

    // Definindo o operador () para que a struct se comporte como uma função
    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;
    }
};

int main() {
    int n = 100; // Tamanho dos vetores
    int a = 5;   // Constante 'a' (modifique para testar diferentes valores)

    // Vetores de entrada gerados aleatoriamente na CPU
    thrust::host_vector<int> h_a(n);
    thrust::host_vector<int> h_b(n);

    // Inicializando os vetores com valores aleatórios
    for (int i = 0; i < n; i++) {
        h_a[i] = rand() % 100;  // Valores aleatórios de 0 a 99
        h_b[i] = rand() % 100;  // Valores aleatórios de 0 a 99
    }

    // Transferindo os vetores para a GPU
    thrust::device_vector<int> d_a = h_a;
    thrust::device_vector<int> d_b = h_b;
    thrust::device_vector<int> d_c(n);  // Vetor de saída

    // Aplicando Saxpy com Thrust
    thrust::transform(d_a.begin(), d_a.end(), d_b.begin(), d_c.begin(), saxpy(a));

    // Copiando o resultado de volta para a CPU
    thrust::host_vector<int> h_c = d_c;

    // Imprimindo os primeiros 10 resultados
    std::cout << "Primeiros 10 resultados da operação z = a * x + y com a = " << a << ":\n";
    for (int i = 0; i < 10; i++) {
        std::cout << "z[" << i << "] = " << h_c[i] << std::endl;
    }

    return 0;
}

Writing saxpy_exercicio.cu


In [4]:
!nvcc -arch=sm_75 -std=c++14 saxpy_exercicio.cu -o exemplo1

In [6]:
!./exemplo1

Primeiros 10 resultados da operação z = a * x + y com a = 5:
z[0] = 501
z[1] = 400
z[2] = 500
z[3] = 522
z[4] = 266
z[5] = 337
z[6] = 509
z[7] = 341
z[8] = 226
z[9] = 396


#### *Calculando a magnitude de um vetor*

In [7]:
%%writefile magnitude_comparacao.cu

#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <chrono>  // Para medir o tempo de execução

// Functor para elevação ao quadrado (transformação unária)
struct square {
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};

// Função para calcular a magnitude na GPU usando Thrust
float magnitude_gpu(thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}

// Função para calcular a magnitude na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for (float x : v) {
        sum_of_squares += x * x;
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    int n = 1 << 20;  // Tamanho do vetor (2^20)

    // Gerando um vetor de floats aleatórios na CPU
    std::vector<float> h_v(n);
    for (int i = 0; i < n; ++i) {
        h_v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Transferindo o vetor para a GPU
    thrust::device_vector<float> d_v = h_v;

    // Medindo o tempo de execução na CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();
    float magnitude_cpu_result = magnitude_cpu(h_v);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> cpu_duration = end_cpu - start_cpu;

    // Medindo o tempo de execução na GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();
    float magnitude_gpu_result = magnitude_gpu(d_v);
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> gpu_duration = end_gpu - start_gpu;

    // Imprimindo os resultados e os tempos de execução
    std::cout << "Magnitude calculada na CPU: " << magnitude_cpu_result << "\n";
    std::cout << "Tempo de execução na CPU: " << cpu_duration.count() << " segundos\n";

    std::cout << "Magnitude calculada na GPU: " << magnitude_gpu_result << "\n";
    std::cout << "Tempo de execução na GPU: " << gpu_duration.count() << " segundos\n";

    return 0;
}

Writing magnitude_comparacao.cu


In [8]:
!nvcc -arch=sm_75 -std=c++14 magnitude_comparacao.cu -o exemplo2

In [9]:
!./exemplo2

Magnitude calculada na CPU: 591.067
Tempo de execução na CPU: 0.0114888 segundos
Magnitude calculada na GPU: 591.177
Tempo de execução na GPU: 0.000720082 segundos


#### *Otimização com Fusion Kernel*

In [13]:
%%writefile variancia_fusion_kernel.cu

#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/reduce.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <chrono>  // Para medir o tempo de execução

// Functor para calcular a variância
struct variance_op {
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;  // Eleva ao quadrado a diferença da média
    }
};

// Functor para a diferença ao quadrado (transformação)
struct diff_square_op {
    float mean;
    diff_square_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        return (x - mean) * (x - mean);
    }
};

// Função para calcular a variância utilizando fusion kernel
float calculate_variance_fusion_kernel(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

// Função para calcular a média
float calculate_mean(const thrust::device_vector<float>& d_vec) {
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    return sum / d_vec.size();
}

// Função para calcular a variância em duas etapas separadas (média e depois variância)
float calculate_variance_two_step(const thrust::device_vector<float>& d_vec) {
    float mean = calculate_mean(d_vec);
    thrust::device_vector<float> diff_squares(d_vec.size());

    // Usando o functor diff_square_op no lugar de lambda
    thrust::transform(d_vec.begin(), d_vec.end(), diff_squares.begin(), diff_square_op(mean));

    float sum_of_squares = thrust::reduce(diff_squares.begin(), diff_squares.end(), 0.0f, thrust::plus<float>());
    return sum_of_squares / d_vec.size();
}

int main() {
    int n = 1 << 20;  // Tamanho do vetor (2^20)

    // Gerando um vetor de floats aleatórios na CPU
    std::vector<float> h_vec(n);
    for (int i = 0; i < n; ++i) {
        h_vec[i] = static_cast<float>(rand()) / RAND_MAX;  // Preenche o vetor com valores aleatórios
    }

    // Transferindo o vetor para a GPU
    thrust::device_vector<float> d_vec = h_vec;

    // Calculando a média (necessário para ambos os métodos)
    float mean = calculate_mean(d_vec);

    // Medindo o tempo de execução com Fusion Kernel
    auto start_fusion = std::chrono::high_resolution_clock::now();
    float variance_fusion = calculate_variance_fusion_kernel(d_vec, mean);
    auto end_fusion = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> fusion_duration = end_fusion - start_fusion;

    // Medindo o tempo de execução com duas etapas
    auto start_two_step = std::chrono::high_resolution_clock::now();
    float variance_two_step = calculate_variance_two_step(d_vec);
    auto end_two_step = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> two_step_duration = end_two_step - start_two_step;

    // Imprimindo os resultados
    std::cout << "Variância com Fusion Kernel: " << variance_fusion << "\n";
    std::cout << "Tempo de execução com Fusion Kernel: " << fusion_duration.count() << " segundos\n";

    std::cout << "Variância com método de duas etapas: " << variance_two_step << "\n";
    std::cout << "Tempo de execução com método de duas etapas: " << two_step_duration.count() << " segundos\n";

    return 0;
}

Overwriting variancia_fusion_kernel.cu


In [14]:
!nvcc -arch=sm_75 -std=c++14 variancia_fusion_kernel.cu -o exemplo3

In [15]:
!./exemplo3

Variância com Fusion Kernel: 0.0833018
Tempo de execução com Fusion Kernel: 0.000270605 segundos
Variância com método de duas etapas: 0.0833018
Tempo de execução com método de duas etapas: 0.000690883 segundos


Para um tamanho de vetor de 2^20, foi possível observar que utilizando a técnica de Fusion Kernel, o tempo de execução foi 3 vezes menor que o tempo de execução utilizando o método de duas etapas